In [ ]:
import socket
import os
import struct
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import time
import random

AES_KEY_SIZE = 32  # AES-256
AES_NONCE_SIZE = 12  # Standard nonce size for AES-GCM
RANDOM_PADDING_MAX = 256  # Max size of random padding

def generate_session_key():
    return get_random_bytes(AES_KEY_SIZE)

def add_random_padding(data):
    padding_length = random.randint(0, RANDOM_PADDING_MAX)
    padding = get_random_bytes(padding_length)
    return data + padding, padding_length

def encrypt_data(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data, padding_length = add_random_padding(plaintext)
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    return nonce + struct.pack("B", padding_length) + ciphertext + tag

def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('localhost', 9999))  # Connect to server on port 9999

    # Send handshake message to server
    client_socket.send(b"HELLO SERVER, PLEASE SEND YOUR ENCRYPTION METHOD")
    message = client_socket.recv(1024)
    print(f"Received from server: {message.decode()}")

    # Receive the session key from the server
    session_key = client_socket.recv(AES_KEY_SIZE)
    print("Session key received from server.")

    # Encrypt and send some data to the server
    plaintext = b"Hello, this is a test message with random padding!"
    encrypted_data = encrypt_data(plaintext, session_key)
    client_socket.send(encrypted_data)

    # Wait for server's response
    response = client_socket.recv(1024)
    print(f"Received from server: {response.decode()}")

    client_socket.close()

if __name__ == "__main__":
    start_client()


Received from server: HELLO SERVER, PLEASE SEND YOUR ENCRYPTION METHOD
Session key received from server.


In [ ]:
import socket
import os
import struct
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256
import random
from sympy import nextprime

AES_KEY_SIZE = 32  # AES-256
AES_NONCE_SIZE = 12  # Standard nonce size for AES-GCM
RANDOM_PADDING_MAX = 256  # Max size of random padding
SESSION_KEY_SIZE = 32  # Size for session keys

# Diffie-Hellman key exchange helper functions
def generate_large_prime(bits=2048):
    random_num = os.urandom(bits // 8)
    return nextprime(int.from_bytes(random_num, 'big'))

def key_exchange(private_key, prime_p, generator_g):
    public_key = pow(generator_g, private_key, prime_p)
    return public_key

def derive_session_key(private_key, other_public_key, prime_p):
    shared_secret = pow(other_public_key, private_key, prime_p)
    session_key = SHA256.new(str(shared_secret).encode()).digest()[:SESSION_KEY_SIZE]
    return session_key

def add_random_padding(data):
    padding_length = random.randint(0, RANDOM_PADDING_MAX)
    padding = get_random_bytes(padding_length)
    return data + padding, padding_length

def encrypt_data(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data, padding_length = add_random_padding(plaintext)
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    print(nonce + struct.pack("B", padding_length) + ciphertext + tag)
    return nonce + struct.pack("B", padding_length) + ciphertext + tag

def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('localhost', 9999))  # Connect to server on port 9999

    # Receive prime and generator from the server
    prime_p, generator_g = client_socket.recv(1024).decode().split('\n')
    prime_p = int(prime_p.split(':')[1].strip())
    generator_g = int(generator_g.split(':')[1].strip())
    
    # Generate private key and compute public key
    bob_private = int.from_bytes(os.urandom(32), 'big') % (prime_p - 2) + 1
    bob_public = key_exchange(bob_private, prime_p, generator_g)

    # Send the client's public key to the server
    client_socket.send(str(bob_public).encode())

    # Receive the server's public key
    alice_public = int(client_socket.recv(1024).decode().strip())

    # Perform Diffie-Hellman key exchange
    session_key = derive_session_key(bob_private, alice_public, prime_p)
    
    print("Session key exchange completed.")

    # Encrypt and send some data to the server
    plaintext = b"this is random padding data with full of paddings!"
    encrypted_data = encrypt_data(plaintext, session_key)
    client_socket.send(encrypted_data)

    # Wait for server's response (optional)
    response = client_socket.recv(1024)
    print(f"Received from server: {response.decode()}")

    client_socket.close()

if __name__ == "__main__":
    start_client()


In [1]:
import socket
import os
import struct
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256
import random
from sympy import nextprime

AES_KEY_SIZE = 32  # AES-256
AES_NONCE_SIZE = 12  # Standard nonce size for AES-GCM
RANDOM_PADDING_MAX = 256  # Max size of random padding
SESSION_KEY_SIZE = 32  # Size for session keys

# Diffie-Hellman key exchange helper functions
def generate_large_prime(bits=2048):
    random_num = os.urandom(bits // 8)
    return nextprime(int.from_bytes(random_num, 'big'))

def key_exchange(private_key, prime_p, generator_g):
    public_key = pow(generator_g, private_key, prime_p)
    return public_key

def derive_session_key(private_key, other_public_key, prime_p):
    shared_secret = pow(other_public_key, private_key, prime_p)
    session_key = SHA256.new(str(shared_secret).encode()).digest()[:SESSION_KEY_SIZE]
    return session_key

def add_random_padding(data):
    padding_length = random.randint(0, RANDOM_PADDING_MAX)
    padding = get_random_bytes(padding_length)
    return data + padding, padding_length

def encrypt_data(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data, padding_length = add_random_padding(plaintext)
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    return nonce + struct.pack("B", padding_length) + ciphertext + tag

def decrypt_data(record, key):
    nonce = record[:AES_NONCE_SIZE]
    padding_length = struct.unpack("B", record[AES_NONCE_SIZE:AES_NONCE_SIZE + 1])[0]
    tag_start = -16
    ciphertext = record[AES_NONCE_SIZE + 1:tag_start]
    tag = record[tag_start:]

    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data = cipher.decrypt_and_verify(ciphertext, tag)
    
    return padded_data[:-padding_length] if padding_length > 0 else padded_data

def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('localhost', 9999))  # Connect to server on port 9999

    # Receive prime and generator from the server
    prime_p, generator_g = client_socket.recv(1024).decode().split('\n')
    prime_p = int(prime_p.split(':')[1].strip())
    generator_g = int(generator_g.split(':')[1].strip())
    
    # Generate private key and compute public key
    bob_private = int.from_bytes(os.urandom(32), 'big') % (prime_p - 2) + 1
    bob_public = key_exchange(bob_private, prime_p, generator_g)

    # Send the client's public key to the server
    client_socket.send(str(bob_public).encode())

    # Receive the server's public key
    alice_public = int(client_socket.recv(1024).decode().strip())

    # Perform Diffie-Hellman key exchange
    session_key = derive_session_key(bob_private, alice_public, prime_p)
    
    print("Session key exchange completed.")

    # Start sending messages in a loop
    while True:
        # Get user input
        message = input("Enter your message (type 'exit' to close): ")

        if message.lower() == 'exit':
            print("Closing connection.")
            client_socket.close()
            break

        # Encrypt and send the message to the server
        encrypted_data = encrypt_data(message.encode(), session_key)
        client_socket.send(encrypted_data)

        # Wait for the server's response
        response = client_socket.recv(1024)
        decrypted_response = decrypt_data(response, session_key)
        print(f"Server Response: {decrypted_response.decode()}")

if __name__ == "__main__":
    start_client()


Session key exchange completed.


Enter your message (type 'exit' to close):  hiii


Server Response: Received your message: hiii


Enter your message (type 'exit' to close):  hiii hiii hiii hiii hiii hiii hiii hiii hiii hiiih iiiih ihiiii hiii hiiii hiiiiiii hiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii hwllo hwloo hello


Server Response: Received your message: hiii hiii hiii hiii hiii hiii hiii hiii hiii hiiih iiiih ihiiii hiii hiiii hiiiiiii hiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii hwllo hwloo hello


Enter your message (type 'exit' to close):  Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a, venenatis vitae, justo. Nullam dictum felis eu pede mollis pretium. Integer tincidunt. Cras dapibus. Vivamus elementum semper nisi. Aenean vulputate eleifend tellus. Aenean leo ligula, porttitor eu, consequat vitae, eleifend ac, enim. Aliquam lorem ante, dapibus in, viverra quis, feugiat a, tellus. Phasellus viverra nulla ut metus varius laoreet. Quisque rutrum. Aenean imperdiet. Etiam ultricies nisi vel augue. Curabitur ullamcorper ultricies nisi. Nam eget dui. Etiam rhoncus. Maecenas tempus, tellus eget condimentum rhoncus, sem quam semper libero

ConnectionResetError: [Errno 104] Connection reset by peer